In [2]:
import numpy as np
from search import *
import queue
import time

## Gates of Light Problem

In the mystical kingdom of Luminaris lies the fortress known as the Gates of Light, guarding treasures of immense power. Protected by an enchantment, the fortress has **G** magical gates, each bound by powerful spells. **L** mystical levers are the keys to unlocking these gates. Each lever can toggle the state of certain gates, and their effects are charted on a mystical map represented by a **LxG** matrix. Only the right combination of lever activations can open all the gates and reveal the treasures.

In the matrix, the rows represent the leavers, while the columns correspond to the magical gates. The value at position **(i, j)** in the matrix is 1 if leaver **i** affects gate **j**, causing it to change state, and 0 if it does not.

For instance if we have 2 leavers and 3 gates, then activations can be respresentate as:
[[0, 1, 1],
[1, 0, 0]]

If the states of the doors are (open, closed, open) and we use the first leaver then the state of the first door will remain the same, while the other two doors will change their states: (open, open, closed).

## 1. Implementation of class GatesOfLightProblem
Your task is to complete the methods in the GatesOfLightProblem class, which extends the SearchProblem class, ensuring that it reflects the Gates of Light problem. You must implement the methods as specified without changing the method signatures or the provided parameters.

In [3]:
class GatesOfLightProblem(SearchProblem):
    """A class that represents Gates O Light Problem"""

    def __init__(self, start, goal, activations):
        self.start = start
        self.goal = goal
        self.activations = activations

    def start_node(self):
        """
        Retrieve the starting node for the search problem.
        Returns:
        - node: The initial state of the problem.
        """

        return self.start

    def is_goal(self, node):
        """
        Check if the given node represents the goal state.

        Args:
        - node: Current state of the problem.
        
        Returns:
        - bool: True if the current state (node) matches the goal state, False otherwise.
        """
        return self.goal == node

    def get_neighbours(self, node):
        """
        Generate and return the neighboring nodes for a given node.

        Args:
        - node: Current state of the problem.
        
        Returns:
        - list of Arcs: A list of `Arc` objects, each representing a transition from the current node to a neighbour.
        
        To track not only states, but actions you would need to pass name for the action when creating a neighbour:
        Arc(node, neighbour, cost, name_of_the_action)
        
        """
        neighbours = []
        for lever, effect in enumerate(self.activations):
            new_state = [node[i] ^ int(effect[i]) for i in range(len(node))]
            neighbours.append(Arc(node, new_state, 1, str(" Apply lever " + str(lever) + " ")))
        return neighbours

    def heuristic(self, node):
        """
        Estimate the distance from the current state to the goal state.

        Args:
        - node: Current state of the problem.

        Returns:
        - float or int: Estimated distance to the goal state.
        """
        return sum(1 for i in range(len(node)) if node[i] != self.goal[i])

## 2. Test Case for GatesOfLightProblem

Below is the implementation of Breadth First Search algorithm that uses closed list to track already explored states. This Algorithm is an extention of the Searcher class from search.py which represent DFS.

In [4]:
class BreadthFirstSearcher(Searcher):
    def __init__(self, problem):
        """
        Initialize the BreadthFirstSearcher.

        Args:
            problem (SearchProblem): The search problem to be solved.
        """
        self.closed = []  # List to keep track of visited nodes
        self.max_frontier_length = 0  # Track the maximum length of the frontier
        super().__init__(problem)  # Initialize the base class with the problem
        self.num_expanded = 0

    def search(self):
        """
        Perform a breadth-first search to find the solution.

        Returns:
            Path: The path from start to goal if a solution is found.
        """
        while not self.empty_frontier():
            # Update the maximum length of the frontier
            self.max_frontier_length = max(self.max_frontier_length, len(self.frontier))

            # Pop the first path from the frontier (FIFO order)
            self.path = self.frontier.pop(0)

            # Add the end node of the current path to the closed list
            self.closed.append(self.path.end())
            self.num_expanded += 1

            # Check if the current path's end node is the goal
            if self.problem.is_goal(self.path.end()):  # Solution found
                self.solution = self.path  # Store the solution found
                return self.path

            else:
                # Get the neighbors of the current end node
                neighbours = self.problem.get_neighbours(self.path.end())
                for arc in neighbours:
                    # Check if the neighbor node has already been visited
                    if arc.to_node in self.closed:
                        continue  # Skip if already visited
                    # Add the new path to the frontier
                    self.add_to_frontier(Path(self.path, arc))

The code below it will generate a random door activation matrix and then use BFS to solve the problem.
The only part is missing is the specification of the start and the goal state. Complete the code and the run the cell.

In [5]:
# assume there are 3 leavers and 4 gates
n_leavers = 3
n_gates = 4

"""Specify the start and the goal for the problem"""
start = [0, 0, 0, 0]  # complete this line
goal = [1, 1, 1, 1]  # complete this line

# gates activation matrix for 3 leavers and 4 gates
test_activations = [[0, 0, 0, 1],
                    [0, 1, 1, 0],
                    [1, 0, 0, 0]]

# creates a problem with the specified start an goal states, and the generated activation funciton
test_gates_problem = GatesOfLightProblem(start, goal, test_activations)

# creates searcher based on BFS
test_searcher = BreadthFirstSearcher(test_gates_problem)

# search for the solution and then print it
test_solution = test_searcher.search()
print(test_solution)

[0, 0, 0, 0]
   -- Apply lever 0 --> [0, 0, 0, 1]
   -- Apply lever 1 --> [0, 1, 1, 1]
   -- Apply lever 2 --> [1, 1, 1, 1]


## 3. GatesOfLightProblem Instance

The code in the cell below generates a gate activation matrix based on you student id. If BFS can't find a solution for the gate activation generated based on your student it, it will increment it ans search again. The only part missing in thois code is the start and the goal states. Complete the corresponding line of code and run the cell.

In [6]:
student_id = 767227544  # Update your student id number, it should have from 7 to 9 digits
seed = student_id

n_gates = 7
n_leavers = 8

"""Specify the start and the goal for the problem"""
start = [0, 0, 0, 0, 0, 0, 0]  # complete this line
goal = [1, 1, 1, 1, 1, 1, 1]  # complete this line

student_solution = None
while student_solution is None:
    np.random.seed(seed)
    print(f"Seed: {seed}")

    #generate gate activations
    student_activations = np.random.randint(0, 2, (n_leavers, n_gates))

    #generate gates of light problem with the generated gate activations
    student_gates_problem = GatesOfLightProblem(start, goal, student_activations)

    # cretae a BFS searcher
    student_searcher = BreadthFirstSearcher(student_gates_problem)

    # search for the solution
    student_solution = student_searcher.search()

    # increment seed
    seed += 1

Seed: 767227544


Complete and run the code in the cell below to print the following information:
- The solution (path from start to goal).
- The number of explored nodes.
- Max length of the fontier.
- The cost of the solution.
- The length of the solution.

You can modify the BreadthFirstSearcher class if you need to introduce new variables to keep track of these details.

In [7]:
print("Solution:", str(student_solution))
print("Number of explored nodes:", student_searcher.num_expanded)
print("Max length of the frontier:", student_searcher.max_frontier_length)
print("The cost of the solution:", student_solution.cost)
print("The length of the soluiton:", len(student_solution))

Solution: [0, 0, 0, 0, 0, 0, 0]
   -- Apply lever 0 --> [0, 0, 0, 1, 1, 0, 0]
   -- Apply lever 2 --> [0, 1, 0, 1, 1, 1, 0]
   -- Apply lever 5 --> [1, 1, 1, 1, 1, 1, 1]
Number of explored nodes: 75
Max length of the frontier: 372
The cost of the solution: 3
The length of the soluiton: 3


## 4. Implementing Informed & Uninformed Search Algorithms

In the cell below, you will find 4 search algorithm classes that you need to complete:

- UniformCostSearcher: Inherits from BreadthFirstSearcher
- GreedySearcher: Inherits from UniformCostSearcher
- AStarSearcher: Inherits from UniformCostSearcher
- IterativeDeepeningAStarSearcher: Inherits from Searcher

You must implement the methods as specified without changing the method signatures or the provided parameters.

Hint: Some search algorithms use a priority queue as the frontier. You can use the built-in PriorityQueue class from Python's queue module for this purpose: pq = queue.PriorityQueue()

In [8]:
class UniformCostSearcher(BreadthFirstSearcher):
    """
    A searcher that implements Uniform Cost Search.

    Inherits from:
        BreadthFirstSearcher: Provides base functionality for search operations.
    """

    def __init__(self, problem):
        """
        Initialize the UniformCostSearcher.

        Args:
            problem (SearchProblem): The search problem to be solved.
        """
        self.pq = None
        #add a order of path @@self.counter, because the pq behaviour in python FIFO order 
        #will be overwritten by the letters' alphabetical order in a path
        #this will impact the intended behaviour of UCS, because when two path
        #have same cost, the newer one will be placed at the front in next iteration
        #which logically should be the older path in the front.
        self.counter = 0 
        super().__init__(problem)

    def initialize_frontier(self):
        """
        Initialize the frontier.
        """
        self.pq = queue.PriorityQueue()
        return

    def add_to_frontier(self, path):
        """
        Add a path to the frontier.

        Args:
            path (Path): The path to be added to the frontier.
        """
        self.counter +=1 #add a order of path
        self.pq.put((path.cost, self.counter, path))
        return

    def empty_frontier(self):
        """
        Check if the frontier is empty.

        Returns:
            bool: True if the frontier is empty, False otherwise.
        """
        return self.pq.empty()

    def search(self):
        """
        Perform the Uniform Cost Search to find the solution.
        
        Returns:
            Path: The path from the start node to the goal node if found, otherwise None.
        """

        while not self.empty_frontier():

            self.max_frontier_length = max(self.max_frontier_length, self.pq.qsize())
            
            cost, counter, self.path = self.pq.get()
            
            self.num_expanded += 1

            self.closed.append(self.path.end())

            if self.problem.is_goal(self.path.end()):
                return self.path
            else:
                neighbours = self.problem.get_neighbours(self.path.end())
                for arc in neighbours:
                    if arc.to_node in self.closed:
                        continue
                    new_path = Path(self.path, arc)
                    self.add_to_frontier(new_path)


In [9]:
class GreedySearcher(UniformCostSearcher):
    """
    A searcher that implements Greedy Best-First Search.

    Inherits from:
        UniformCostSearcher: Provides base functionality for search operations.
    """

    def __init__(self, problem):
        """
        Initialize the GreedySearcher.

        Args:
            problem (SearchProblem): The search problem to be solved.
        """
        super().__init__(problem)

    def add_to_frontier(self, path):
        """
        Add a path to the frontier.
        Args:
            path (Path): The path to be added to the frontier.
        """
        h = self.problem.heuristic(path.end())
        self.pq.put((h, self.counter, path))
        return


In [10]:
class AStarSearcher(UniformCostSearcher):
    """
    A searcher that implements A* Star Search.

    Inherits from:
        UniformCostSearcher: Provides base functionality for search operations.
    """

    def __init__(self, problem):
        """
        Initialize the GreedySearcher.

        Args:
            problem (SearchProblem): The search problem to be solved.
        """
        super().__init__(problem)

    def add_to_frontier(self, path):
        """
        Add a path to the frontier.

        Args:
            path (Path): The path to be added to the frontier.
        """
        g = path.cost
        h = self.problem.heuristic(path.end())
        f = g + h
        self.pq.put((f, self.counter, path))
        return

In [11]:
class IterativeDeepeningAStarSearcher(Searcher):
    """A class representing ID A* (acts like IDS with heuristic instead of the depth)
        search method is overwritten to iterate over different thresholds
        * values - array to store the values (cost + heuristic) of the neighbors
        * threshold - the current threshold
        * start - initial path to start the search with the new threshold
    """

    def __init__(self, problem):
        """
        Initialize the GreedySearcher.

        Args:
            problem (SearchProblem): The search problem to be solved.
        """
        self.stack = None
        self.threshold = 0
        self.start = problem.start_node()
        super().__init__(problem)


    def initialize_frontier(self):
        """
        Initialize the frontier.
        """
        self.stack = []
        return

    def empty_frontier(self):
        """
        Check if the frontier is empty.

        Returns:
            bool: True if the frontier is empty, False otherwise.
        """
        return self.stack == []

    def add_to_frontier(self, path):
        """
        Add a path to the frontier.

        Args:
            path (Path): The path to be added to the frontier.
        """
        g = path.cost
        h = self.problem.heuristic(path.end())
        f = g + h
        self.stack.append((path, f))
        return

    def search(self):
        
        """
        Perform the Uniform Cost Search to find the solution.
        Returns:
            Path: The path from the start node to the goal node if found, otherwise None.
        """
        h_start = self.problem.heuristic(self.start)
        f_start = h_start
        self.threshold = f_start
        
        infinity = float('inf')

        while self.threshold < infinity:
            
            min_threshold = infinity
            
            """
            since in the beginning, we add initial node to the pq already
            when we invoked super().__init__(problem), however, if we re-enter
            the inner while loop (A*), we need to add it back again, so the frontier
            will not be empty as well as ensures that we can start from the beginning
            node again
            """
            if self.empty_frontier():
                self.add_to_frontier(Path(self.start))

            while not self.empty_frontier():
                self.max_frontier_length = max(self.max_frontier_length, len(self.stack))
                
                path, f = self.stack.pop()
                      
                self.num_expanded += 1

                if self.problem.is_goal(path.end()):
                    return path

                neighbours = self.problem.get_neighbours(path.end())
                for arc in neighbours:
                    new_path = Path(path, arc)
                    g = new_path.cost
                    h = self.problem.heuristic(new_path.end())
                    f = g + h

                    if f <= self.threshold:
                        self.add_to_frontier(new_path)
                    else:
                        min_threshold = min(min_threshold, f)

            self.threshold = min_threshold
        return

## 5. Analysis of Search Algorithms' Performance

Run each algorithm listed in the table below on the problem defined by the following gate activations:

In [12]:
seed = 15
np.random.seed(seed)

n_gates = 9
n_leavers = 9

#generated gate activations
gate_activations = np.random.randint(0, 2, (n_leavers, n_gates))

print(gate_activations)

start = [0, 0, 0, 0, 0, 0, 0, 0, 0]
goal = [1, 1, 1, 1, 1, 1, 1, 1, 1]

gates_problem = GatesOfLightProblem(start, goal, gate_activations)

# searcher = IterativeDeepeningAStarSearcher(gates_problem) #IDA
# searcher = UniformCostSearcher(gates_problem) #UCS
# searcher = BreadthFirstSearcher(gates_problem) #BFS
# searcher = AStarSearcher(gates_problem) #A* #Greedy
searcher = GreedySearcher(gates_problem) #Greedy

#ignore time for class instance construction
start_time = time.time()

solution = searcher.search()

end_time = time.time()
execution_time = end_time - start_time
print(f"Method executed in: {execution_time:.6f} seconds")
print("Solution:", str(solution))
print("Number of explored nodes:", searcher.num_expanded)
print("Max length of the frontier:", searcher.max_frontier_length)
print("The cost of the solution:", solution.cost)
print("The length of the soluiton:", len(solution))

[[0 1 0 1 1 0 0 1 1]
 [1 1 0 1 1 1 1 1 1]
 [0 0 0 1 0 1 1 1 1]
 [0 1 0 1 1 0 1 0 0]
 [1 1 1 0 1 0 0 1 0]
 [1 0 1 0 1 0 0 0 1]
 [0 1 0 0 1 0 1 0 0]
 [0 1 1 1 0 1 0 1 1]
 [1 0 0 1 1 1 0 0 0]]
Method executed in: 0.004000 seconds
Solution: [0, 0, 0, 0, 0, 0, 0, 0, 0]
   -- Apply lever 1 --> [1, 1, 0, 1, 1, 1, 1, 1, 1]
   -- Apply lever 5 --> [0, 1, 1, 1, 0, 1, 1, 1, 0]
   -- Apply lever 8 --> [1, 1, 1, 0, 1, 0, 1, 1, 0]
   -- Apply lever 2 --> [1, 1, 1, 1, 1, 1, 0, 0, 1]
   -- Apply lever 6 --> [1, 0, 1, 1, 0, 1, 1, 0, 1]
   -- Apply lever 4 --> [0, 1, 0, 1, 1, 1, 1, 1, 1]
   -- Apply lever 5 --> [1, 1, 1, 1, 0, 1, 1, 1, 0]
   -- Apply lever 0 --> [1, 0, 1, 0, 1, 1, 1, 0, 1]
   -- Apply lever 7 --> [1, 1, 0, 1, 1, 0, 1, 1, 0]
   -- Apply lever 5 --> [0, 1, 1, 1, 0, 0, 1, 1, 1]
   -- Apply lever 8 --> [1, 1, 1, 0, 1, 1, 1, 1, 1]
   -- Apply lever 3 --> [1, 0, 1, 1, 0, 1, 0, 1, 1]
   -- Apply lever 6 --> [1, 1, 1, 1, 1, 1, 1, 1, 1]
Number of explored nodes: 81
Max length of the frontier: 54

In the table below record the cost of the solution, the number of explored nodes, max length of the frontier, and the time it took to find the solution for each algorithm. If an algorithm runs for more than 1 minute terminate it and mark as Timeout.


| Search Algorithms       | Cost | Explored | Max Frontier Length | Time              |
|-------------------------|------|----------|---------------------|-------------------|
| Breadth First Search    | 7    | 79212    | 181441              | Timeout (383.20s) |
| Uniform Cost Search     | 7    | 79212    | 181441              | Timeout (393.67s) |
| Greedy Search           | 13   | 81       | 544                 | 0.004593s         |
| A Star Search           | 7    | 5572     | 18797               | 3.497990s         |
| IDA Star Search         | 7    | 182470   | 41                  | 5.703394s         |

Answer the following questions:
    

#### Question 1: Compare the performance of the BFS and UCS algotihms. Is there anything that surprised you? (max 150 words)
Answer: 

The performance of Breadth-First Search (BFS) and Uniform Cost Search (UCS) is quite similar in this scenario, with both algorithms finding the optimal solution with a cost of 7. Each explored 79,212 nodes and reached a maximum frontier length of 181,441, but both timed out due to extensive time requirements that are 383.20 seconds for BFS and 393.67 seconds for UCS. This outcome is expected since UCS behaves like BFS when all path costs are uniform, as in this Gate of Light problem. The slight difference in timing is odd, as UCS is typically expected to be quicker, or at least identical in performance when all arc costs are the same. One possible explanation could be associated with UCS's use of a priority queue, which may involve extra library invocations and the overhead in handling costs. 

#### Question 2: Summarise the performance of the Informed Search Startegies. Is it what you expected? (max 300 words)
Answer: 

The performance of three informed search strategies, Greedy Search, A* Search, and IDA* Search varies significantly in their generated statistic, but it is as expected.

Greedy Search is the fastest, taking only 0.004593 seconds to find a solution with a cost of 13, exploring 81 nodes, and having a maximum frontier length of 544. This speed comes from prioritizing nodes closest to the goal based on the heuristic. However, it doesn’t guarantee finding the least-cost path because it ignores the actual cost of the path, leading to a suboptimal solution, as reflected by the higher cost compared to A* and IDA* searches.

A* Search took 3.497990 seconds, finding an optimal solution with a cost of 7, exploring 5572 nodes, and with a maximum frontier length of 18,797. A* Search balances the heuristic with the actual path cost from the start node, ensuring an optimal solution but requiring more time and memory, as indicated by the larger frontier and longer computation time.

IDA* Search, the slowest at 5.703394 seconds, also found an optimal solution with a cost of 7, exploring 182,470 nodes but with a much smaller maximum frontier length of 41. This is expected since IDA* is more memory-efficient, using depth-first search approach with iterative deepening, where the depth is controlled by a heuristic. The smaller frontier reflects its memory efficiency, but the need to re-explore paths due to the iterative deepening approach results in a larger number of explored nodes and longer time, showcasing the trade-off between memory usage and speed.


## 6. Graph Search

The code in the cell below implements a search problem using an explicit graph:

In [13]:
class GraphProblem(SearchProblem):
    """A search problem from an explicit graph."""

    def __init__(self, nodes, arcs, start=None, goals=set(), hmap={},
                 positions=None, show_costs=True):
        """ A search problem consists of:
        * list or set of nodes
        * list or set of arcs
        * start node
        * list or set of goal nodes
        * hmap: dictionary that maps each node into its heuristic value.
        """
        self.neighbours = {}
        self.nodes = nodes
        for node in nodes:
            self.neighbours[node] = []
        self.arcs = arcs
        for arc in arcs:
            self.neighbours[arc.from_node].append(arc)
        self.start = start
        self.goals = goals
        self.hmap = hmap

    def start_node(self):
        """returns start node"""
        return self.start

    def is_goal(self, node):
        """is True if node is a goal"""
        return node in self.goals

    def get_neighbours(self, node):
        """returns the neighbors of node (a list of arcs)"""
        return self.neighbours[node]

    def heuristic(self, node):
        """Gives the heuristic value of node n.
        Returns 0 if not overridden in the hmap."""
        if node in self.hmap:
            return self.hmap[node]
        else:
            return 0

    def __repr__(self):
        """returns a string representation of the search problem"""
        res = ""
        for arc in self.arcs:
            res += f"{arc}.  "
        return res

Run the cell below to create an instance of GraphProblem with 28 nodes:

In [14]:
graph_problem = GraphProblem(
    {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
     'X', 'Y', 'Z', 'AA', 'AB'},
    [
        Arc('A', 'B', 5), Arc('A', 'C', 7), Arc('B', 'D', 4), Arc('B', 'E', 6), Arc('C', 'F', 8), Arc('C', 'G', 5),
        Arc('D', 'H', 3), Arc('D', 'I', 7), Arc('E', 'J', 2), Arc('E', 'K', 9), Arc('F', 'L', 6), Arc('F', 'M', 4),
        Arc('G', 'N', 8), Arc('G', 'O', 3), Arc('H', 'P', 7), Arc('I', 'Q', 5), Arc('J', 'R', 6), Arc('K', 'S', 8),
        Arc('L', 'T', 4), Arc('M', 'U', 9), Arc('N', 'V', 2), Arc('O', 'W', 7), Arc('P', 'X', 5), Arc('Q', 'Y', 8),
        Arc('R', 'Z', 3), Arc('S', 'AA', 6), Arc('T', 'AB', 7), Arc('U', 'A', 9), Arc('V', 'B', 5), Arc('W', 'C', 6),
        Arc('X', 'D', 8), Arc('Y', 'E', 4), Arc('Z', 'F', 7), Arc('AA', 'G', 5), Arc('AB', 'H', 6)
    ],
    start='A',
    goals={'AB'},
    hmap={
        'A': 12, 'B': 8, 'C': 10, 'D': 9, 'E': 7, 'F': 6,
        'G': 11, 'H': 8, 'I': 10, 'J': 5, 'K': 12, 'L': 7,
        'M': 6, 'N': 9, 'O': 11, 'P': 10, 'Q': 8, 'R': 7,
        'S': 12, 'T': 6, 'U': 10, 'V': 5, 'W': 8, 'X': 7,
        'Y': 9, 'Z': 11, 'AA': 6, 'AB': 0,
    })

# searcher = IterativeDeepeningAStarSearcher(graph_problem) #IDA
# searcher = UniformCostSearcher(graph_problem) #UCS
# searcher = BreadthFirstSearcher(graph_problem) #BFS
# searcher = AStarSearcher(graph_problem) #A*
searcher = GreedySearcher(graph_problem) #Greedy

#ignore time for class instance construction
start_time = time.time()

solution = searcher.search()

end_time = time.time()
execution_time = end_time - start_time
print(f"Method executed in: {execution_time:.6f} seconds")

print("Solution:", str(solution))
print("Number of explored nodes:", searcher.num_expanded)
print("Max length of the frontier:", searcher.max_frontier_length)
print("The cost of the solution:", solution.cost)
print("The length of the soluiton:", len(solution))

Method executed in: 0.000120 seconds
Solution: A --> C --> F --> L --> T --> AB
Number of explored nodes: 13
Max length of the frontier: 7
The cost of the solution: 32
The length of the soluiton: 5


In the table below, record the following for each algorithm applied to the graph problem:

- Cost of the solution
- Number of explored nodes
- Maximum length of the frontier
- Time taken to find the solution

If an algorithm runs for more than 1 minute, terminate it and mark it as "Timeout."



| Search Algorithms       | Cost | Explored | Max Frontier Length | Time      |
|-------------------------|------|----------|---------------------|-----------|
| Breadth First Search    | 32   | 28       | 8                   | 0.000069s |
| Uniform Cost Search     | 32   | 27       | 8                   | 0.000136s |
| Greedy Search           | 32   | 13       | 7                   | 0.000099s |
| A Star Search           | 32   | 23       | 8                   | 0.000234s |
| IDA Star Search         | 32   | 143      | 4                   | 0.000266s |

#### Question: For each algorithm compare its performnace in tasks 5 & 6. If there are any differences explain them. (max 300 words)
Answer:

Breadth-First Search (BFS) has drastic difference in performance is due to Task 5’s larger state space and uniform costs, which resulted in massive exploration and led to timed out. Task 6’s Graph problem with simpler structure and less state space allowed BFS to quickly explore fewer nodes and solve the problem efficiently, and surprisingly for both tasks, BFS were able to find optimal result even when arcs' cost are different in Task 6.

Uniform Cost Search (UCS) similar to BFS, UCS in Task 5 had to explore a vast number of nodes due to the uniform cost structure, leading to a timeout. Task 6’s smaller graph with varied arc costs allowed UCS to perform efficiently indicated by less nodes explored (time is extremely similar, yet longer, might due to the same reason mentioned Task 5 Q1 again), demonstrating its typical advantage of UCS relative to BFS when path costs differ.

Greedy Search performs consistently fast in both tasks, but its suboptimality is more evident in Task 5, where the higher cost indicates a less efficient path. Task 6’s simpler problem structure led to quicker and an optimal result.

A* consistently finds optimal solutions but is more computationally expensive relative Greedy Search same as in the task 5. Task 5’s larger state space and uniform costs increased the exploration required. Task 6’s smaller graph allowed A* to perform more efficiently, though still slower than Greedy Search.

IDA* is slower than other search methods in both tasks due to its iterative deepening approach, which requires re-exploration of nodes from beginning each time went back from outer while loop when threshold needed to change. Task 5’s large state space amplified this issue, leading to high explored nodes count, whereas Task 6’s smaller problem allowed quicker, though still less efficient compared to other algorithms.

### References
List any resources you used to complete this assigment

[1] W3School: https://www.w3schools.com/python/: As I am a beginner in Python, I use this tutorial extensively to learn the syntax and how class and inheritance work in Python.

[2] ChatGPT: Again, the resource on W3School is limited, thus, I used ChatGPT to explore the behaviour of various methods that involved in built-in Python libraries (i.e. priorityQueue).

[3] Lecture slides: Yes, definitely used lecture slide prepared by Anna, they are very helpful in referring back.